In [9]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time
import xml.etree.ElementTree as ET
from PIL import Image
import glob
# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 Physical GPUs, 1 Logical GPUs


# 把所有img和xml放在mAP/ 下即可（mAP/img 和 mAP/xml）

In [10]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('424detect.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
#model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'c', 1: 'gzp', 2: 'jdk', 3: 'jzp', 4: 'ljt',5:'nnh',6:'wsz',7:'xjp',8:'ym'}

tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [-50.796833, -50.796833,  50.7

# detect no_match的图片，在自动生成的detect/no_match里

In [11]:
confidence = 0.35
if not os.path.exists('detect/no_match/'): 
    os.makedirs((os.path.join('detect','no_match')))
no_match_path = 'results/images/True_wrong/'
for imgs in os.listdir(no_match_path):
    name = imgs.rsplit('.jpg')[0]
    #for img in os.listdir('413/true_wrong'):
    # load image
    image = read_image_bgr('img/' + name + '.jpg')

    # copy to draw on
    draw = image.copy()
    #draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    # draw = cv2.cvtColor(draw,)

    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)

    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    #print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    # visualize detections
    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted so we can break
        if score < confidence:
            break

        color = label_color(label)

        b = box.astype(int)
        draw_box(draw, b, color=color)

        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)

    #plt.figure(figsize=(15, 15))
    #plt.axis('off')
    #plt.imshow(draw)
    #plt.show()
    cv2.imwrite('detect/no_match/' + name +'.jpg', draw)
print('done')

done


# no_match图片的label，放在自动生成的label/no_match里

In [12]:
no_match_path = 'results/images/True_wrong/'
if not os.path.exists('label/no_match/'): 
    os.makedirs((os.path.join('label','no_match')))
xml_dst = 'xml/'
for imgs in os.listdir(no_match_path):
    name = imgs.rsplit('.jpg')[0]
    
    
    tree = ET.parse(xml_dst + name + '.xml')
    root = tree.getroot()
    imgfile = 'img/' + name +'.jpg'
    im = cv2.imread(imgfile)
    for object in root.findall('object'):
        object_name = object.find('name').text
        Xmin = int(object.find('bndbox').find('xmin').text)
        Ymin = int(object.find('bndbox').find('ymin').text)
        Xmax = int(object.find('bndbox').find('xmax').text)
        Ymax = int(object.find('bndbox').find('ymax').text)
        color = (4, 250, 7)
        cv2.rectangle(im, (Xmin, Ymin), (Xmax, Ymax), color, 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(im, object_name, (Xmin, Ymin - 7), font, 0.5, (6, 230, 230), 2)

    cv2.imwrite('label/no_match/' + name +'.jpg', im)
print('done')

# 把label和detect合并，放在自动生成的combine/no_match

In [13]:
img1_dst = 'detect/no_match/'
img2_dst = 'label/no_match/'
if not os.path.exists('combine/no_match/'): 
    os.makedirs((os.path.join('combine','no_match')))
for img in os.listdir(img1_dst):
    img1, img2 = Image.open(img1_dst + '/' + img), Image.open(img2_dst +'/' + img)
    size1, size2 = img1.size, img2.size

    joint = Image.new('RGB', (size1[0]+size2[0], size1[1]))
    loc1, loc2 = (0, 0), (size1[0], 0)
    joint.paste(img1, loc1)
    joint.paste(img2, loc2)
    joint.save('combine/no_match/' + img)
print('done')

done


# 给combine后的图片调整大小（默认1280*720）

In [14]:
combine_path = 'combine/no_match/'
for img in os.listdir(combine_path):
    height = 720
    width = 1280
    top, bottom, left, right = (0, 0, 0, 0)
    image = cv2.imread(combine_path + img, cv2.IMREAD_UNCHANGED)

     #获取图像尺寸
    h, w, _ = image.shape

     #对于长宽不相等的图片，找到最长的一边
    longest_edge = max(h, w)    

     #计算短边需要增加多上像素宽度使其与长边等长
    if h < longest_edge:
        dh = longest_edge - h
        top = dh // 2
        bottom = dh - top
    elif w < longest_edge:
        dw = longest_edge - w
        left = dw // 2
        right = dw - left
    else:
         pass 

     #RGB颜色
    BLACK = [0, 0, 0]

     #给图像增加边界，是图片长、宽等长，cv2.BORDER_CONSTANT指定边界颜色由value指定
    constant = cv2.copyMakeBorder(image, top , bottom, left, right, cv2.BORDER_CONSTANT, value = BLACK)

     #调整图像大小并返回
    cv2.resize(constant, (height, width))
    cv2.imwrite(combine_path + img,constant) 
print('done')

done


# 生成video，放在自动生成的video/no_match/

In [15]:
combine_path = 'combine/no_match/' 
if not os.path.exists('video/no_match/'): 
    os.makedirs((os.path.join('video','no_match')))
img_array = []
for filename in glob.glob(combine_path + '*.jpg'):
    
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
     
 
out = cv2.VideoWriter('video/no_match/no_match_424.avi',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
print('done')

done
